IFT6135-H2019
Assignment 1 Programming part
Multilayer Parceptrons and Convolutional Neural Networks

Alejandra
Amlin
Charles
Georgina

In [53]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Helping functions

In [54]:
def softmax(x):
    shiftx = x - np.max(x)
    exps=np.exp(shiftx)
    y=exps/np.sum(exps)
    return y

def relu (x):
    y=np.maximum(0, x)
    return y

#function taken from IFT6093 cours
def onehot(y, n_classes):
    o = np.zeros(shape=(y.shape[0], n_classes))
    for i in range(y.shape[0]):
        o[i, int(y[i])] = 1
    return o

In [56]:
#This class implementation is inspired from the NN implemented in cours IFT6093
class NN(object):
    
    
    def __init__(self, input_dim, n, output_dim,hidden_dims=(1024,2048),n_hidden=2,mode='train',
                 datapath=None,model_path=None):
        
        self.hd1 = hidden_dims[0] 
        self.hd2 = hidden_dims[1]
        self.n_hidden = n_hidden
        self.n = n
        self.outd = output_dim
        self.W1 = np.zeros(size=(hidden_dims[0], input_dim))
        self.b1 = np.zeros(hidden_dims[0])
        self.W2 = np.zeros(size=(hidden_dims[1], hidden_dims[0]))
        self.b2 = np.zeros(hidden_dims[1])
        self.W3 = np.zeros(size=(output_dim, hidden_dims[1]))
        self.b3 = np.zeros(output_dim)
        self.parameters = [self.b1, self.W1, self.b2, self.W2, self.b3, self.W3]
        
        
    def initialize_weights_normal(self,n_hidden,dims):
        
        self.W1 = np.random.normal(loc=0.0, scale=1.0, size=(self.hd1, self.n))
        self.W2 = np.random.normal(loc=0.0, scale=1.0, size=(self.hd2, self.hd1))
        self.W3 = np.random.normal(loc=0.0, scale=1.0, size=(self.outd, self.hd2))
    
    
    def initialize_weights_glorot(self,n_hidden,dims):
        
        dl1 = np.sqrt(6/(self.hd1 + self.hd2))
        dl2 = np.sqrt(6/(self.hd2 + self.outd))
        self.W1 = np.random.uniform(low=(-dl1), high=dl1, size=(self.hd1, self.n))
        self.W2 = np.random.uniform(low=(-dl1), high=dl1, size=(self.hd2, self.hd1))
        self.W3 = np.random.uniform(low=(-dl2), high=dl2, size=(self.outd, self.hd2))
    

    def forward(self,input,labels):
        
        a1 = np.dot (self.W1, x) + self.b1 
        h1 = activation (h_a)
        a2 = np.dot (self.W2, x) + self.b2
        h2 = activation (a2)
        oa = np.dot (self.W3, x) + self.b3
        os = softmax (oa, axis=0)
               
        return a1, h1, a2, h2, oa, os
    
        

    #Method inspired from NN implemented in cours IFT6093
    def activation (self,input):
        return (input > 0) * input
    

    #Method inspired from NN implemented in cours IFT6093
    def loss (self,prediction,os):
        return (y * (-np.log(os))).sum()
    

    def softmax (self,input,axis=1):
        shiftx = x - np.max (x, axis=axis, keepdims=True)
        exps = np.exp (shiftx)
        y = exps / exps.sum (axis=axis, keepdims=True)
        return y
    

    def backward(self,cache, x, y,a1, h1, a2, h2, oa, os, weight_decay=0):
    
        grad_oa = os - y
        grad_W3 = np.outer (grad_oa, h2) + weight_decay * self.W3
        grad_b3 = grad_oa
        grad_h2 = np.dot (self.W3.T, grad_oa)
        grad_a2 = (a2 > 0) * grad_h2
        grad_W2 = np.outer (grad_a2, h1) + weight_decay * self.W2
        grad_b2 = grad_a2 
        grad_h1 = np.dot (self.W2.T, grad_oa)
        grad_a1 = (a1 > 0) * grad_h1
        grad_W1 = np.outer (grad_a1, x) + weight_decay * self.W1
        grad_b1 = grad_a1
   
        return grad_W3, grad_b3, grad_W2, grad_b2, grad_W1, grad_b1



    def update(self,average_grads, mu):
    
        for p, grad in zip(self.parameters, average_grads):
            p -= mu * grad
    
        #self.b1= self.b1+mu*grad_b1
        #self.W1= self.W1+mu*grad_W1
        #self.b2= self.b2+mu*grad_b2
        #self.W2= self.W2+mu*grad_W2
        #self.b3= self.b3+mu*grad_b3
        #self.W3= self.W3+mu*grad_W3
        
       

    def train_SGD(self, x, y, mb_size=100, learning_rate=1e-1, weight_decay=0):
        
        for i in range (0, x.shape[0], mb_size):
        
            xi = x[i:(i+mb_size)]
            yi = y[i:(i+mb_size)]
            
            sum_grads = [np.zeros(shape=p.shape) for p in self.parameters]
            losses = 0
            
            a1, h1, a2, h2, oa, os = self.forward(self,xi,yi)
            grad = self.backward(self, xi, yi, a1, h1, a2, h2, oa, os, cache=none)
            
            sum_grads = [g1 + g2 for g1, g2 in zip(grad, sum_grads)]
            loss = self.loss(self, yi, os)
            losses += loss                          
            average_grads = [g / xi.shape[0] for g in sum_grads]
            average_loss = losses / xi.shape[0]
            self.parameters = update_parms(average_grads, mu)
        
            return average_loss

In [ ]:
def test(self):
    .
    .